<a href="https://colab.research.google.com/github/tannerhs/tannerhs.github.io/blob/master/cleaning_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#read tmx file in
!pip install translate-toolkit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.3/744.3 kB 10.3 MB/s eta 0:00:00


In [32]:
from translate.storage.tmx import tmxfile
import translate.storage.tmx
from translate.storage.tmx import tmxunit
from translate.storage.base import TranslationUnit

input_file_1="/content/14 320 - Korean - FamilyHistory - Recent.tmx"
input_file_2="/content/14 320 - Korean - FamilyHistory - Recent.tmx"

source_language="en-us"
target_language="kor-KR"
tmx_list=[]

with open(input_file_1,"rb") as file: #rb is needed, otherwise things freak out if you don't remove the first line with <?xml...
  xml=file.read()
  tmx=tmxfile(xml,source_language,target_language, encoding="utf-8")
for node in tmx.unit_iter():
  tmx_list.append([node.source,node.target])

In [69]:
def remove_segments(tu):
  if((len(tu[0])<5) or (len(tu[1]) <5)):  #remove empty and almost empty segments
    return None
  #remove segments that are too long, too imbalanced (source or target much longer than other), or too much just numbers
  if (tu[0].count(" ")<2) or (tu[1].count(" ")<2): #remove units with fewer than 3 words
    return None
  elif (tu[0].count(" ")>99) or (tu[0].count(" ")>99): #remove units with more than 100 words
    return None
  elif( len(tu[0])/len(tu[1])>=2 or len(tu[1])/len(tu[0])>=2) :
    return None
  elif ((len(re.findall('[0-9]',tu[0]))/len(tu[0]))>0.4) or (len(re.findall('[0-9]',tu[1]))/len(tu[1])>0.4):
    return None  #if too much of string is numbers (more than 40%) just remove the tu
  else :
    return tu

# def remove_hex_characters(tu):
#   re.subn("&#([a-zA-Z0-9])+ ","",tu[0])



def clean(tu):
    # alignment and other cleaning
    tu[1]=re.subn(r'\n|\r|&#xd;?|&#x2028;?|&#x2029;?|&.*;|\t','', tu[1])[0]
    tu[0]=re.subn(r'\n|\r|&#xd;?|&#x2028;?|&#x2029;?|&.*;|\t','', tu[0])[0]
    tu[0]=re.subn(r'\\n|\\r|\\t|\\:|\\f|\%|\\.+|%(.*)%|(<.*>)+|({.*})+','', tu[0])[0]
    tu[1]=re.subn(r'\\n|\\r|\\t|\\:|\\f|\%|\\.+|%(.*)%|(<.*>)+|({.*})+','', tu[1])[0]

    # handle uneven {}, [],(),<>

    #normalize whitespace-- messes up alignment!
    # tu[0]=re.subn(r' +',' ', tu[0])[0]
    # tu[1]=re.subn(r' +',' ', tu[1])[0]


    # tu[0]=re.subn(r'(\\.+)+','',tu[0])[0]
    # tu[1]=re.subn(r'(\\.+)+','',tu[1])[0]
    # tu[0]=re.subn(r'(\\.\s+)+','',tu[0])[0]
    # tu[1]=re.subn(r'(\\.\s+)+','',tu[1])[0]
    # tu[0]=re.subn(r'(\\:)+','',tu[0])[0]
    # tu[1]=re.subn(r"(\\:)+",'',tu[1])[0]
    # tu[0]=re.subn(r'\\:','',tu[0])[0]
    # tu[1]=re.subn(r'\\:','',tu[1])[0]

    #handle angle brackets
    tu[0]=re.subn(r'(<.*>)+','',tu[0])[0]
    tu[1]=re.subn(r'(<.*>)+','',tu[1])[0]
    tu[0]=re.subn(r'(<)+',"", tu[0])[0]
    tu[1]=re.subn(r'(<)+',"", tu[1])[0]
    tu[0]=re.subn(r'(>)+',"", tu[0])[0]
    tu[1]=re.subn(r'(>)+',"", tu[1])[0]

    #
    # tu[0]=re.subn(r'(#.*\s+)+',"", tu[0])[0]
    # tu[1]=re.subn(r'(#.*\s+)+',"", tu[1])[0]
    # tu[0]=re.subn(r'(&.*\s+)+',"",tu[0])[0]
    # tu[1]=re.subn(r'(&.*\s+)+',"",tu[1])[0]

    #normalize quotes
    tu[0]=re.subn(r'‘|’','\'',tu[0])[0]
    tu[1]=re.subn(r'‘|’','\'',tu[1])[0]
    tu[0]=re.subn(r'“|”','\"',tu[0])[0]
    tu[1]=re.subn(r'“|”','\"',tu[1])[0]


    if remove_segments(tu) is None:
      return None

    # tu=remove_hex_characters(tu)
    return tu

In [70]:
#iterate over all units
import re
import string

def normalize(str):
  return re.subn('[%s]' % re.escape(string.punctuation), '', str)[0]


# iter=tmx.unit_iter()
end_not_reached=True
i=0
count=0
clean_tmx_list=[]
clean_tmx_set=set()


src_file=open("/content/source.txt","w")
tgt_file=open("/content/target.txt", "w")

while (count<200000) and (i<len(tmx_list)):
  tu=tmx_list[i]
  tu=clean(tu)  #update tu to be the cleaned version
  if (tu is not None):
    clean_tmx_list.append([tu[0],tu[1]])
    count=count+1
  i=i+1


for tu in clean_tmx_list:
  tu_src_normal = normalize(tu[0])
  tu_tgt_normal = normalize(tu[1])
  tu_normal_tuple=(tu_src_normal,tu_tgt_normal)
  if (tu_src_normal!=tu_tgt_normal) and (tu_normal_tuple not in clean_tmx_set):
    clean_tmx_set.add((normalize(tu[0]),normalize(tu[1])))
    src_file.write(tu[0]+"\n")
    tgt_file.write(tu[1]+"\n")
src_file.close()
tgt_file.close()

In [ ]:
#now write to file


In [21]:
# type(tmx)

# print(tmx.getsourcelanguage())  #todo, incorrect format in tmx, need to convert to iso
# print(tmx.gettargetlanguage())
# clean_tmx=TMXFile()
# clean_tmx=tmxfile(inputfile=None,sourcelanguage="en",targetlanguage="kor")
new_tu=tmxunit(source="hi")
new_tu.settarget("안녕"*100)
# clean_tmx.addunit(new_tu)

print(type(new_tu))
print(new_tu.target)
print(new_tu.source)

<class 'translate.storage.tmx.tmxunit'>
안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕안녕
hi


In [127]:
test="귀하 지역의 상담자(들)\:"
re.subn(r'\\:','',test)

('귀하 지역의 상담자(들)', 1)

In [ ]:
test=map(tuple,[["a","b"],["c","d"],["a","b"]])